### [GZIP file format specification version 4.3](http://zlib.org/rfc-gzip.html)

* a 10-byte header, containing a magic number (1f 8b), compression id (08 for DEFLATE), file flags, a 32-bit timestamp, compression flags and operating system id.
```
1f 8b
08
00 
e4 18 af 4b
00
03
```

* optional extra headers denoted by file flags, such as the original filename
* a body, containing a DEFLATE-compressed payload

* an 8-byte footer, containing a CRC-32 checksum and the length of the original uncompressed data, modulo 2^32
```
17 7d 42 4a
00 b8 64 01 
```

#### FLG (FLaGs)

    This flag byte is divided into individual bits as follows:

    bit 0   FTEXT
    bit 1   FHCRC
    bit 2   FEXTRA
    bit 3   FNAME
    bit 4   FCOMMENT
    bit 5   reserved
    bit 6   reserved
    bit 7   reserved

    If FTEXT is set, the file is probably ASCII text. This is an optional indication, which the compressor may set by checking a small amount of the input data to see whether any non-ASCII characters are present. In case of doubt, FTEXT is cleared, indicating binary data. For systems which have different file formats for ascii text and binary data, the decompressor can use FTEXT to choose the appropriate format. We deliberately do not specify the algorithm used to set this bit, since a compressor always has the option of leaving it cleared and a decompressor always has the option of ignoring it and letting some other program handle issues of data conversion.

    If FHCRC is set, a CRC16 for the gzip header is present, immediately before the compressed data. The CRC16 consists of the two least significant bytes of the CRC32 for all bytes of the gzip header up to and not including the CRC16. [The FHCRC bit was never set by versions of gzip up to 1.2.4, even though it was documented with a different meaning in gzip 1.2.4.]

    If FEXTRA is set, optional extra fields are present, as described in a following section.

    If FNAME is set, an original file name is present, terminated by a zero byte. The name must consist of ISO 8859-1 (LATIN-1) characters; on operating systems using EBCDIC or any other character set for file names, the name must be translated to the ISO LATIN-1 character set. This is the original name of the file being compressed, with any directory components removed, and, if the file being compressed is on a file system with case insensitive names, forced to lower case. There is no original file name if the data was compressed from a source other than a named file; for example, if the source was stdin on a Unix system, there is no file name.

    If FCOMMENT is set, a zero-terminated file comment is present. This comment is not interpreted; it is only intended for human consumption. The comment must consist of ISO 8859-1 (LATIN-1) characters. Line breaks should be denoted by a single line feed character (10 decimal).

    Reserved FLG bits must be zero. 

####  OS (Operating System)
    This identifies the type of file system on which compression took place. This may be useful in determining end-of-line convention for text files. The currently defined values are as follows:

      0 - FAT filesystem (MS-DOS, OS/2, NT/Win32)
      1 - Amiga
      2 - VMS (or OpenVMS)
      3 - Unix
      4 - VM/CMS
      5 - Atari TOS
      6 - HPFS filesystem (OS/2, NT)
      7 - Macintosh
      8 - Z-System
      9 - CP/M
     10 - TOPS-20
     11 - NTFS filesystem (NT)
     12 - QDOS
     13 - Acorn RISCOS
    255 - unknown



In [10]:
GZIP_FLAG = ("ASCII_Text", "CRC16", "Extra_Field", "File_Name", "Comment", "Reserved", "Reserved", "Reserved")

GZIP_OS = {
    b'\x00': "FAT filesystem (MS-DOS, OS/2, NT/Win32)",
    b'\x01': "Amiga",
    b'\x02': "VMS (or OpenVMS)",
    b'\x03': "Unix",
    b'\x04': "VM/CMS",
    b'\x05': "Atari TOS",
    b'\x06': "HPFS filesystem (OS/2, NT)",
    b'\x07': "Macintosh",
    b'\x08': "Z-System",
    b'\x09': "CP/M",
    b'\x0a': "TOPS-20",
    b'\x0b': "NTFS filesystem (NT)",
    b'\x0c': "QDOS",
    b'\x0d': "Acorn RISCOS",
    b'\xff': "unknown",
}


GZIP = (
    ("magic number", 2, ("Fixed", b'\x1f\x8b', "0x1f 0x8b")),
    ("compression method", 1, ("Fixed", b'\x08', "Deflate")),
    ("flag", 1, ("flag", GZIP_FLAG)),
    ("modification time", 4, ("integer", "little", "time")),
    ("extra flag", 1, ("dict", {b'\x02': "maximum compression", b'\04': "fastest algorithm"})),
    ("os", 1, ("dict", GZIP_OS)),
    ("extra field", ("flag", 2, (2, "little")), "hex"),
    ("file name", ("flag", 3, b'\x00'), "string"),
    ("comment", ("flag", 4, b'\x00'), "string"),
    ("CRC16", ("flag", 1, 2), "hex"),
    ("compressed data", -8, "hex"),
    ("CRC32", 4, "hex"),
    ("input size", 4, ("integer", "little")),
)